In [1]:
from action_package import ROBOT
import  RPi.GPIO as GPIO
import os
import time 

speed = 30

# infrared sensor
SensorRight = 16  
SensorLeft  = 12

# ultrasonic sensor
TRIG = 20 # output
ECHO = 21 # input

def setup():
    GPIO.setwarnings(False)
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(SensorRight,GPIO.IN)
    GPIO.setup(SensorLeft,GPIO.IN)
    GPIO.setup(TRIG, GPIO.OUT)
    GPIO.setup(ECHO, GPIO.IN)

def get_dist():
    '''
    utrasonic distance detect
    '''
    GPIO.output(TRIG, 0) # initailly low
    time.sleep(0.000002)
    
    GPIO.output(TRIG, 1) # high 10ms
    time.sleep(0.00001)
    GPIO.output(TRIG, 0) # set back to low

    # start time
    while GPIO.input(ECHO) == 0:
        a = 0
    time1 = time.time()

    # end time
    while GPIO.input(ECHO) == 1:
        a = 1
    time2 = time.time()
    
    during = time2 - time1   
    
    time.sleep(0.02)
    return round(during * 340 / 2 * 100) # calculate distance (in cm)


if __name__ == "__main__":
    robot = ROBOT()
    setup()
    try:
        while True:
            SR = GPIO.input(SensorRight)
            SL = GPIO.input(SensorLeft)
            dist = get_dist()
            #print(f"dist = {dist} cm")

            # SL & SR = 1 (not detected), 0 (detected)
            if dist > 10 and SL == True and SR == True: 
                robot.t_up(speed,0) # go forward
            elif dist > 10 and SL == True and SR == False:
                robot.turnLeft(speed,0.5)
            elif dist > 10 and SL == False and SR == True:
                robot.turnRight(speed,0.5)
            else:
                robot.t_down(speed,1) # go backward
    except KeyboardInterrupt:
        robot.t_stop(0) # stop
        GPIO.cleanup()
